<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-准备数据" data-toc-modified-id="1-准备数据-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1 准备数据</a></span><ul class="toc-item"><li><span><a href="#1.1-加载文本" data-toc-modified-id="1.1-加载文本-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1.1 加载文本</a></span></li><li><span><a href="#1.2-将整个文本以句子拆分" data-toc-modified-id="1.2-将整个文本以句子拆分-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>1.2 将整个文本以句子拆分</a></span></li></ul></li><li><span><a href="#2-jieba分词" data-toc-modified-id="2-jieba分词-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2 jieba分词</a></span></li><li><span><a href="#3-创建MinHashForest及MinHash对象" data-toc-modified-id="3-创建MinHashForest及MinHash对象-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3 创建MinHashForest及MinHash对象</a></span><ul class="toc-item"><li><span><a href="#3.1-自定义MinHash函数" data-toc-modified-id="3.1-自定义MinHash函数-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>3.1 自定义MinHash函数</a></span></li><li><span><a href="#3.2-minhash降维及建forest" data-toc-modified-id="3.2-minhash降维及建forest-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>3.2 minhash降维及建forest</a></span></li><li><span><a href="#3.3-索引到目前为止添加的所有key，并使它们可搜索" data-toc-modified-id="3.3-索引到目前为止添加的所有key，并使它们可搜索-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>3.3 索引到目前为止添加的所有key，并使它们可搜索</a></span></li></ul></li><li><span><a href="#4-查询top-3近似最近邻居" data-toc-modified-id="4-查询top-3近似最近邻居-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>4 查询top-3近似最近邻居</a></span></li><li><span><a href="#5-总结" data-toc-modified-id="5-总结-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>5 总结</a></span></li></ul></div>

Action1要求：    
使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt    

针对某句话进行Query，查找Top-3相似的句子

In [1]:
import re
import jieba 
from datasketch import MinHash, MinHashLSHForest

# 1 准备数据
## 1.1 加载文本

In [2]:
# 读入微博文本
with open('weibos.txt', 'r', encoding='utf-8') as f:
    contents = f.read()

In [3]:
# 读入自定义的停用词
with open('stopword.txt', 'r', encoding='utf-8') as f:
    # 直接readlines()仍然会有换行符，不如直接整个读取后split
    stopwords = f.read().split()

In [4]:
stopwords

['的', '是', '，', '“', '”', '：']

## 1.2 将整个文本以句子拆分

In [5]:
# 不可见字符\u200b的数量
contents.count('\u200b')

7

In [6]:
# 去除不可见字符\u200b和空格和换行符
contents = contents.replace('\u200b', '').replace(' ', '').replace('\n', '')

In [7]:
# 按句子拆分
sentences = re.split(pattern='[#。！？]', string=contents)
len(sentences)

41

In [8]:
# 统计列表中空字符串数量
sentences.count('')

2

In [9]:
# 号称去除列表中空字符串最快最简单的方法 == [x for x in your_list if x]，但速度更快
# 第一个参数为None，默认会去除序列中所有值为假的元素
sentences = list(filter(None, sentences))
len(sentences)

39

In [10]:
sentences[:2]

['斯科拉里愿意执教国足', '上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用']

# 2 jieba分词

In [11]:
documents = []
for s in sentences:
    temp = ' '.join([w for w in jieba.cut(s) if w not in stopwords])
    documents.append(temp)        

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/dk/4d4_y1kn1js69jdc1n1xyjtr0000gn/T/jieba.cache
Loading model cost 0.713 seconds.
Prefix dict has been built successfully.


# 3 创建MinHashForest及MinHash对象

## 3.1 自定义MinHash函数

In [12]:
def get_minhash(doc):
    """为每个句子生成一个MinHash实例"""
    m  = MinHash(num_perm=128)
    for word in doc:
        m.update(word.encode('utf-8'))
    return m

## 3.2 minhash降维及建forest

In [13]:
forest = MinHashLSHForest(num_perm=128, l=8)
minhash_list = []

for i,doc in enumerate(documents):
    m = get_minhash(doc)
    minhash_list.append(m)
    # 将minhash对象加入到MinHashLSHForest中
    # key和index对应，方便后面查询
    forest.add(key=i, minhash=m)

## 3.3 索引到目前为止添加的所有key，并使它们可搜索
必备操作，不可少

In [14]:
forest.index()

# 4 查询top-3近似最近邻居

In [15]:
# 要查询的句子的索引号
query_index = 8

# 要查询Top-k的近似最近邻，因为有一个是原句，所以看top3时，K为4
K = 4

In [16]:
# 查询索引号对应的原句
query_str = sentences[query_index]
query_str

'辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利'

In [17]:
# 获取top-3近似最近邻居的keys
neighbor_keys = forest.query(minhash_list[query_index], k=K)
neighbor_keys

[8, 35, 29, 38]

In [18]:
for k in neighbor_keys:
    if k == query_index:
        continue
    query_doc = set(documents[query_index])
    neighbor = set(documents[k])
    real_jaccard = len(query_doc.intersection(neighbor)) / len(query_doc.union(neighbor))
    minhash_value = minhash_list[query_index].jaccard(minhash_list[k])
    print("要查询的索引{}的原句:\n{}\n".format(query_index, query_str))
    print("它的近似最近邻居索引:{}, 对应句子:\n{}\n".format(k, sentences[k]))
    print("查询(index={})与邻居(index={})的jaccard相似度:{}\n".format(query_index, k, minhash_value))
    print("二者的真实jaccard相似度:{}".format(real_jaccard))
    print('*' * 80)
    print('')

要查询的索引8的原句:
辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利

它的近似最近邻居索引:35, 对应句子:
国足输给叙利亚后，里皮坐不住了，直接辞职了难怪有网友说，爱护生命，远离男足

查询(index=8)与邻居(index=35)的jaccard相似度:0.21875

二者的真实jaccard相似度:0.19230769230769232
********************************************************************************

要查询的索引8的原句:
辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利

它的近似最近邻居索引:29, 对应句子:
尤其是最后一句话，看好中国队的潜力，这句话真是太鼓舞人心啦

查询(index=8)与邻居(index=29)的jaccard相似度:0.046875

二者的真实jaccard相似度:0.038461538461538464
********************************************************************************

要查询的索引8的原句:
辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利

它的近似最近邻居索引:38, 对应句子:
国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职

查询(index=8)与邻居(index=38)的jaccard相似度:0.2265625

二者的真实jaccard相似度:0.17777777777777778
********************************************************************************



# 5 总结

1. 对比了jieba.cut和jieba.posseg.cut，前者只分词，而后者在分词时还会标注词性，网上有人说二者分词结果会有差异，但我随机找了几个句子分词对比了下，单词结果没有发现太大区别。
2. MinHashForest找到的jaccard相似度的值，与真实jaccard相似度确实很接近。
3. index=8的与邻居的近似jaccard相似度:index=38 > index=35(0.2266 > 0.2188)，但真实jaccard相似度:index=38 < index=35 (0.1778 < 0.1923)，可见当近似jaccard相似度大于其他邻居的相似度时，真实jaccard相似度不一定也大于其他邻居。所以近似jaccard相似度是top1时，不一定真实相似度是第一，所以是近似最近邻查找。
4. 找到个去除列表中空字符串的函数`filter()`，很好用。